## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_trimmed = application_df.copy()
application_df_trimmed = application_df_trimmed.drop(['EIN', 'NAME'], axis=1)

In [3]:
# Determine the number of unique values in each column.
application_df_trimmed.nunique(axis=0)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_type_count = application_df_trimmed['APPLICATION_TYPE'].value_counts()
app_type_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# create a list of application types to be replaced for values less than 500
# use the variable name `application_types_to_replace`
application_types_to_replace = []
for i in range(len(app_type_count)):
    if app_type_count[i] < 500:
        application_types_to_replace.append(app_type_count.index[i])

# Replace in dataframe
for app in application_types_to_replace:
    application_df_trimmed['APPLICATION_TYPE'] = application_df_trimmed['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_trimmed['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df_trimmed['CLASSIFICATION'].value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts > 100
classification_count.loc[lambda x : x > 100]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []
for i in range(len(classification_count)):
    if classification_count[i] < 500:
        classifications_to_replace.append(classification_count.index[i])

# Replace in dataframe
for cls in classifications_to_replace:
    application_df_trimmed['CLASSIFICATION'] = application_df_trimmed['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df_trimmed['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
# (AFFILIATION, USE_CASE, CLASSIFICATION, USE_CASE, ORGANIZATION, INCOME_AMT SPECIAL_CONSIDERATIONS)
application_df_noncat = pd.get_dummies(data=application_df_trimmed, columns=['APPLICATION_TYPE', 'AFFILIATION',
                                                     'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION',
                                                     'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'])
application_df_noncat.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df_noncat['IS_SUCCESSFUL']
X = application_df_noncat.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Using Random Forest to understand feature importance

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=1)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Accuracy Score : 0.711603498542274
Classification Report
              precision    recall  f1-score   support

           0       0.70      0.68      0.69      4010
           1       0.72      0.74      0.73      4565

    accuracy                           0.71      8575
   macro avg       0.71      0.71      0.71      8575
weighted avg       0.71      0.71      0.71      8575



In [ ]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf_model.feature_importances_
# We can sort the features by their importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.4024979604739238, 'ASK_AMT'),
 (0.13260364385470497, 'AFFILIATION_CompanySponsored'),
 (0.12008482429648537, 'AFFILIATION_Independent'),
 (0.029660794498396495, 'APPLICATION_TYPE_T5'),
 (0.02675568962096524, 'ORGANIZATION_Association'),
 (0.022990501097703408, 'APPLICATION_TYPE_T10'),
 (0.021019878105442865, 'CLASSIFICATION_C2100'),
 (0.018917073702858778, 'ORGANIZATION_Trust'),
 (0.018395180540084235, 'APPLICATION_TYPE_T19'),
 (0.01764906365986777, 'CLASSIFICATION_C2000'),
 (0.016421525866620833, 'APPLICATION_TYPE_T4'),
 (0.01586612339461281, 'APPLICATION_TYPE_T3'),
 (0.01338361088401685, 'CLASSIFICATION_C1000'),
 (0.012636404680482341, 'CLASSIFICATION_C7000'),
 (0.01181797658571914, 'USE_CASE_ProductDev'),
 (0.011092936618495891, 'APPLICATION_TYPE_T6'),
 (0.010504185950338668, 'USE_CASE_Preservation'),
 (0.010116654210775056, 'CLASSIFICATION_C1200'),
 (0.00927533033231979, 'INCOME_AMT_0'),
 (0.009094199754143755, 'CLASSIFICATION_Other'),
 (0.008149605329072126, 'CLASSIFICATION_C3

# Using Keras Tuner

In [12]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=44))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [15]:
!pip install keras_tuner

In [16]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

Using TensorFlow backend


In [17]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 01m 24s]
val_accuracy: 0.7300291657447815

Best val_accuracy So Far: 0.7304956316947937
Total elapsed time: 00h 26m 10s


In [18]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 1,
 'num_layers': 3,
 'units_0': 9,
 'units_1': 7,
 'units_2': 3,
 'units_3': 3,
 'units_4': 5,
 'units_5': 1,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [19]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5750 - accuracy: 0.7305 - 875ms/epoch - 3ms/step
Loss: 0.5750102996826172, Accuracy: 0.7304956316947937
